**Identificar como o tempo está distribuido no espaço**


#Instalando bibliotecas

In [1]:
 !pip install 'fsspec>=0.3.3'

     |████████████████████████████████| 136 kB 4.3 MB/s 


In [2]:
!pip install unidecode

     |████████████████████████████████| 235 kB 4.3 MB/s 


#Importando Bibliotecas e arquivos do drive

In [3]:
from unidecode import unidecode
import dask.dataframe as dd
import pandas as pd
import plotly.express as px
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_validate, GridSearchCV, cross_val_predict
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold

/usr/local/lib/python3.7/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:17: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  "Since version 1.0, "


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!ls "/content/drive/My Drive"

 ATESTADO_MATRICULA__GRADUACAO2014057.pdf
 Certificados
'Colab Notebooks'
 Contas.gsheet
 Curriculo
 ImersaoDev_Alura
'Iniciação Científica'
'Meet Recordings'
 Obra
 UFOP
'Voluta '
'WhatsApp Image 2022-03-17 at 10.17.42.jpeg'
 Wizard


##Lendo os dados

In [6]:
df = dd.read_csv('/content/drive/MyDrive/Colab Notebooks/IC/eccomerce/database/dataframes/olist_new_df.csv')

In [7]:
df.columns

Index(['order_id', 'order_item_id', 'product_id', 'seller_id', 'price',
       'freight_value', 'payment_type', 'payment_installments',
       'payment_value', 'customer_unique_id', 'product_category_name',
       'product_weight_g', 'product_length_cm', 'product_height_cm',
       'product_width_cm', 'costumer_lat', 'costumer_lng', 'seller_lat',
       'seller_lng', 'days_to_delivery', 'early_delivery', 'day_shipping',
       'month_shipping', 'year_shipping', 'day_order_approved',
       'month_order_approved', 'year_order_approved'],
      dtype='object')

In [8]:
df = df.compute()

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111237 entries, 0 to 111236
Data columns (total 27 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   order_id               111237 non-null  float64
 1   order_item_id          111237 non-null  float64
 2   product_id             111237 non-null  float64
 3   seller_id              111237 non-null  float64
 4   price                  111237 non-null  float64
 5   freight_value          111237 non-null  float64
 6   payment_type           111237 non-null  float64
 7   payment_installments   111237 non-null  float64
 8   payment_value          111237 non-null  float64
 9   customer_unique_id     111237 non-null  float64
 10  product_category_name  111237 non-null  float64
 11  product_weight_g       111237 non-null  float64
 12  product_length_cm      111237 non-null  float64
 13  product_height_cm      111237 non-null  float64
 14  product_width_cm       111237 non-nu

#Visualizações do tempo no espaço





##Mapas de calor 


In [ ]:
fig = px.density_mapbox(df, lat='costumer_lat', lon='costumer_lng', z='days_to_delivery', radius=10,
                        center=dict(lat=0, lon=180), zoom=0, height=1000,
                        mapbox_style="stamen-terrain")
fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
fig = px.density_mapbox(df, lat='seller_lat', lon='seller_lng', z='days_to_delivery', radius=10,
                        center=dict(lat=0, lon=180), zoom=0, height=1000,
                        mapbox_style="stamen-terrain")
fig.show()

Output hidden; open in https://colab.research.google.com to view.

##Adicionando costumer city e costumer state ao dataframe


In [10]:
df_aux = dd.read_csv('/content/drive/MyDrive/Colab Notebooks/IC/eccomerce/database/dataframes/olist_geolocated_transactions.csv')

In [11]:
df_aux = df_aux.compute()

In [12]:
df_aux.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 117729 entries, 0 to 14207
Data columns (total 43 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       117729 non-null  object 
 1   order_item_id                  117729 non-null  float64
 2   product_id                     117729 non-null  object 
 3   seller_id                      117729 non-null  object 
 4   shipping_limit_date            117729 non-null  object 
 5   price                          117729 non-null  float64
 6   freight_value                  117729 non-null  float64
 7   payment_sequential             117726 non-null  float64
 8   payment_type                   117726 non-null  object 
 9   payment_installments           117726 non-null  float64
 10  payment_value                  117726 non-null  float64
 11  review_id                      116753 non-null  object 
 12  review_score                   

In [13]:
df_aux.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,payment_sequential,payment_type,payment_installments,...,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,costumer_lat,costumer_lng,seller_lat,seller_lng
0,00010242fe8c5a6d1ba2dd792cb16214,1.0,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29,1.0,credit_card,2.0,...,598.0,4.0,650.0,28.0,9.0,14.0,-21.763186,-41.310265,-22.497188,-44.127324
1,130898c0987d1801452a8ed92a670612,1.0,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-07-05 02:44:11,55.9,17.96,1.0,boleto,1.0,...,598.0,4.0,650.0,28.0,9.0,14.0,-17.884803,-51.717167,-22.497188,-44.127324
2,532ed5e14e24ae1f0d735b91524b98b9,1.0,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2018-05-23 10:56:25,64.9,18.33,1.0,credit_card,2.0,...,598.0,4.0,650.0,28.0,9.0,14.0,-19.913964,-43.976106,-22.497188,-44.127324
3,6f8c31653edb8c83e1a739408b5ff750,1.0,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-08-07 18:55:08,58.9,16.17,1.0,credit_card,3.0,...,598.0,4.0,650.0,28.0,9.0,14.0,-25.520334,-49.124988,-22.497188,-44.127324
4,7d19f4ef4d04461989632411b7e588b9,1.0,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-08-16 22:05:11,58.9,13.29,1.0,credit_card,4.0,...,598.0,4.0,650.0,28.0,9.0,14.0,-20.655379,-43.776331,-22.497188,-44.127324


In [14]:
df_customer =  pd.DataFrame(df_aux['customer_state'], columns=['customer_state'])

In [15]:
df_customer['customer_city'] = df_aux['customer_city']

In [20]:
df_customer['order_id'] = df['order_id']

In [18]:
df_customer['seller_city'] = df_aux['seller_city']

In [21]:
df_customer['seller_state'] = df_aux['seller_state']

In [22]:
df_customer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 117729 entries, 0 to 14207
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   customer_state  117729 non-null  object 
 1   customer_city   117729 non-null  object 
 2   order_id        117729 non-null  float64
 3   seller_city     117729 non-null  object 
 4   seller_state    117729 non-null  object 
dtypes: float64(1), object(4)
memory usage: 5.4+ MB


In [23]:
df_customer.head()

,customer_state,customer_city,order_id,seller_city,seller_state
0,RJ,campos dos goytacazes,0.0,volta redonda,SP
1,GO,jatai,7014.0,volta redonda,SP
2,MG,belo horizonte,30559.0,volta redonda,SP
3,PR,sao jose dos pinhais,40954.0,volta redonda,SP
4,MG,conselheiro lafaiete,45875.0,volta redonda,SP


In [24]:
df_customer.isnull().sum()

customer_state    0
customer_city     0
order_id          0
seller_city       0
seller_state      0
dtype: int64

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111237 entries, 0 to 111236
Data columns (total 27 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   order_id               111237 non-null  float64
 1   order_item_id          111237 non-null  float64
 2   product_id             111237 non-null  float64
 3   seller_id              111237 non-null  float64
 4   price                  111237 non-null  float64
 5   freight_value          111237 non-null  float64
 6   payment_type           111237 non-null  float64
 7   payment_installments   111237 non-null  float64
 8   payment_value          111237 non-null  float64
 9   customer_unique_id     111237 non-null  float64
 10  product_category_name  111237 non-null  float64
 11  product_weight_g       111237 non-null  float64
 12  product_length_cm      111237 non-null  float64
 13  product_height_cm      111237 non-null  float64
 14  product_width_cm       111237 non-nu

In [26]:
db = pd.merge(df, df_customer, how='inner', on='order_id')

In [27]:
db.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 196606 entries, 0 to 196605
Data columns (total 31 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   order_id               196606 non-null  float64
 1   order_item_id          196606 non-null  float64
 2   product_id             196606 non-null  float64
 3   seller_id              196606 non-null  float64
 4   price                  196606 non-null  float64
 5   freight_value          196606 non-null  float64
 6   payment_type           196606 non-null  float64
 7   payment_installments   196606 non-null  float64
 8   payment_value          196606 non-null  float64
 9   customer_unique_id     196606 non-null  float64
 10  product_category_name  196606 non-null  float64
 11  product_weight_g       196606 non-null  float64
 12  product_length_cm      196606 non-null  float64
 13  product_height_cm      196606 non-null  float64
 14  product_width_cm       196606 non-nu

##Malhas

In [29]:
#malha estados
import json

with open('/content/drive/MyDrive/Colab Notebooks/IC/eccomerce/database/dataframes/estados.json') as response:
    counties = json.load(response)


In [30]:
#malha cidades
import json

with open('/content/drive/MyDrive/Colab Notebooks/IC/eccomerce/database/dataframes/cidades.json') as response:
    cidades = json.load(response)


###Pré processamento 

In [31]:
#replace uppercase
for feature in cidades["features"]:
    feature["properties"]["name"] = (feature["properties"]["name"].lower())

In [32]:
#remove accents
for feature in cidades["features"]:
    feature["properties"]["name"] = unidecode(feature["properties"]["name"])

In [33]:
#verificando se funcionou
cidades["features"]

##Visualização para Estado

In [28]:
#Customer
import plotly.express as px

fig = px.choropleth_mapbox(db, geojson=counties, color='days_to_delivery', range_color=(0, 16),
                           locations='customer_state', featureidkey="properties.SIGLA", #vincula location com a sigla presente no json
                           center={"lat": -20.000, "lon": -60.000},
                           mapbox_style="carto-positron", zoom=3)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

NameError: ignored

In [ ]:
fig = px.choropleth_mapbox(db, geojson=counties, color='days_to_delivery', range_color=(0, 16),
                           locations='seller_state', featureidkey="properties.SIGLA", #vincula location com a sigla presente no json
                           center={"lat": -20.000, "lon": -60.000},
                           mapbox_style="carto-positron", zoom=3)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

##Visualização para cidades
 

###Mapa

In [ ]:
import plotly.express as px

fig = px.choropleth_mapbox(db, geojson=cidades, color='days_to_delivery', range_color=(0, 16),
                           locations='customer_city', featureidkey="properties.name", #vincula location com a sigla presente no json
                           center={"lat": -20.000, "lon": -60.000},
                           mapbox_style="carto-positron", zoom=3)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()